In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    dap_accumulator = {}  # لتجميع DAP لكل حالة X-RAY

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            modality = getattr(ds, "Modality", "").upper()
            study_desc = getattr(ds, "StudyDescription", "").lower()
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            k = 0.015
            msv = 0

            # --- CT معالجة
            if modality == "CT":
                if dlp > 0:
                    if "head" in study_desc or "brain" in study_desc:
                        k = 0.0021
                    elif "neck" in study_desc:
                        k = 0.0059
                    elif "chest" in study_desc:
                        k = 0.014
                    elif "abdomen" in study_desc or "pelvis" in study_desc:
                        k = 0.015
                    msv = dlp * k
                else:
                    length = float(getattr(ds, "TotalCollimationWidth", 0))
                    dlp = ctdi * length
                    msv = dlp * k

            # --- X-RAY / CR / DR معالجة DAP
            elif modality in ["CR", "DR", "DX", "XRAY", "X-RAY"]:
                dap = 0

                # محاولة قراءة DAP من التاجات
                for tag in [(0x0018, 0x1405), (0x0018, 0x9323)]:
                    try:
                        val = float(ds.get(tag).value)
                        dap += val
                    except:
                        continue

                # نحفظ كل DAP في dict مؤقت للمريض عشان نجمّعهم
                patient_key = (name, date_obj.date(), getattr(ds, "PatientID", ""), getattr(ds, "AccessionNumber", ""))
                if patient_key not in dap_accumulator:
                    dap_accumulator[patient_key] = 0
                dap_accumulator[patient_key] += dap

                # تحديد k حسب المنطقة
                if "head" in study_desc or "brain" in study_desc:
                    k = 0.0031
                elif "chest" in study_desc:
                    k = 0.017
                elif "abdomen" in study_desc or "pelvis" in study_desc:
                    k = 0.019
                else:
                    k = 0.015

                dlp = dap_accumulator[patient_key]
                msv = dlp * k

            matched_key = None
            for existing in existing_keys:
                if is_same_person(name, existing[0]) and existing[1] == date_obj.date():
                    matched_key = existing
                    break
            key = matched_key if matched_key else (
                name, date_obj.date(), getattr(ds, "PatientID", ""), getattr(ds, "AccessionNumber", "")
            )

            img = None
            if 'PixelData' in ds:
                arr = ds.pixel_array
                if arr.max() > 0:
                    arr = (arr / arr.max() * 255).astype('uint8')
                else:
                    arr = arr.astype('uint8')
                img_pil = Image.fromarray(arr)
                if img_pil.mode not in ["L", "RGB"]:
                    img_pil = img_pil.convert("L")
                img_pil.thumbnail((400, 400))
                img = ImageTk.PhotoImage(img_pil)

            if key not in temp_cases:
                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "kFactor": k,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "PatientID": getattr(ds, "PatientID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Images": [],
                    "Path": path,
                    "Modality": modality,
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

            if img is not None:
                temp_cases[key]["Images"].append(img)

            hl7_msg = convert_to_hl7_from_table(temp_cases[key])
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{temp_cases[key]['PatientID']}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())

    # حساب AccumulatedDose و DosePerYear
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()



In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    grouped_files = {}  # تجميع الملفات حسب (الاسم، التاريخ، ID) لتجميع الحالة

    for path in files:
        try:
            ds = pydicom.dcmread(path, stop_before_pixels=True)
            name = str(getattr(ds, "PatientName", "Unknown"))
            date_str = getattr(ds, "StudyDate", "00000000")
            date_obj = datetime.strptime(date_str, "%Y%m%d") if date_str else datetime.now()
            patient_id = getattr(ds, "PatientID", "")
            accession = getattr(ds, "AccessionNumber", "")

            key = (name, date_obj.date(), patient_id, accession)
            if key not in grouped_files:
                grouped_files[key] = []
            grouped_files[key].append(path)
        except:
            continue

    for key, paths in grouped_files.items():
        name, date, patient_id, accession = key
        modality = ""
        dlp = 0
        ctdi = 0
        msv = 0
        k = 0.015
        study_desc = ""
        images = []

        total_dap = 0
        xray_detected = False
        ct_detected = False

        for path in paths:
            try:
                ds = pydicom.dcmread(path)

                modality = getattr(ds, "Modality", "").upper()
                study_desc = getattr(ds, "StudyDescription", "").lower()

                if modality == "CT":
                    ct_detected = True
                    dlp_val = float(getattr(ds, "DLP", 0))
                    if dlp_val > 0:
                        dlp = max(dlp, dlp_val)
                    ctdi = float(getattr(ds, "CTDIvol", 0))

                elif modality in ["CR", "DR", "DX", "XRAY"]:
                    xray_detected = True
                    for tag in [(0x0018, 0x1405), (0x0018, 0x115E)]:
                        try:
                            val = float(ds.get(tag).value)
                            total_dap += val
                        except:
                            continue

                # تحميل الصور إن وجدت
                if 'PixelData' in ds:
                    arr = ds.pixel_array
                    if arr.max() > 0:
                        arr = (arr / arr.max() * 255).astype('uint8')
                    else:
                        arr = arr.astype('uint8')
                    img_pil = Image.fromarray(arr)
                    if img_pil.mode not in ["L", "RGB"]:
                        img_pil = img_pil.convert("L")
                    img_pil.thumbnail((400, 400))
                    images.append(ImageTk.PhotoImage(img_pil))
            except:
                continue

        if ct_detected:
            # تقدير k حسب منطقة الجسم
            if "head" in study_desc or "brain" in study_desc:
                k = 0.0021
            elif "neck" in study_desc:
                k = 0.0059
            elif "chest" in study_desc:
                k = 0.014
            elif "abdomen" in study_desc or "pelvis" in study_desc:
                k = 0.015
            else:
                k = 0.015
            if dlp == 0:
                dlp = ctdi * 100  # fallback
            msv = dlp * k

        elif xray_detected:
            if "head" in study_desc or "brain" in study_desc:
                k = 0.0031
            elif "chest" in study_desc:
                k = 0.017
            elif "abdomen" in study_desc or "pelvis" in study_desc:
                k = 0.019
            else:
                k = 0.015
            dlp = total_dap
            msv = total_dap * k

        key_full = key
        if key_full not in temp_cases:
            data_dict = {
                "Name": name,
                "Date": datetime.combine(date, datetime.min.time()),
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "kFactor": k,
                "Sex": "",
                "DOB": "",
                "PatientID": patient_id,
                "Accession": accession,
                "Images": images,
                "Path": paths[0],
                "Modality": modality,
                "Dataset": None
            }
            temp_cases[key_full] = data_dict

        hl7_msg = convert_to_hl7_from_table(temp_cases[key_full])
        hl7_filename = f"{HL7_DIR}/{name}_{date.strftime('%Y%m%d')}_{patient_id}.hl7"
        with open(hl7_filename, "w") as f:
            f.write(hl7_msg)

    all_data.extend(temp_cases.values())

    # ✅ حساب AccumulatedDose و DosePerYear
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()



In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    grouped_by_case = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            pid = str(getattr(ds, "PatientID", ""))
            study_date = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(study_date, "%Y%m%d")
            except:
                date_obj = datetime.now()

            case_key = (pid, date_obj.date())
            if case_key not in grouped_by_case:
                grouped_by_case[case_key] = []
            grouped_by_case[case_key].append((path, ds))

        except Exception as e:
            print("Read error:", e)

    for (pid, date), dicom_list in grouped_by_case.items():
        name = "Unknown"
        modality = ""
        dlp = 0
        ctdi = 0
        total_dap = 0
        k = 0.015
        msv = 0
        selected_img = None

        for path, ds in dicom_list:
            modality = getattr(ds, "Modality", "").upper()
            name = str(getattr(ds, "PatientName", "Unknown"))

            if modality == "CT":
                dlp += float(getattr(ds, "DLP", 0))
                ctdi += float(getattr(ds, "CTDIvol", 0))
                desc = getattr(ds, "SeriesDescription", "").lower()
                if dlp == 0 and "protocol" in desc and 'PixelData' in ds:
                    try:
                        arr = ds.pixel_array
                        if arr.dtype != 'uint8':
                            arr = (arr / arr.max() * 255).astype('uint8')
                        img = Image.fromarray(arr).convert("L")
                        text = pytesseract.image_to_string(img)
                        match = re.search(r"total\s*dlp\s*[:=]?[\s\n]*(\d+\.?\d*)", text, re.IGNORECASE)
                        if match:
                            dlp = float(match.group(1))
                    except Exception as e:
                        print("OCR error:", e)

            elif modality in ["CR", "DR", "DX", "X-RAY"]:
                # DAP من التاج
                for elem in ds.iterall():
                    if elem.tag == (0x0018, 0x115E):
                        try:
                            total_dap += float(elem.value)
                        except:
                            pass

            # اختيار صورة واحدة للعرض فقط
            if selected_img is None and 'PixelData' in ds:
                arr = ds.pixel_array
                if arr.dtype != 'uint8':
                    arr = (arr / arr.max() * 255).astype('uint8')
                img_pil = Image.fromarray(arr).convert("L")
                img_pil.thumbnail((400, 400))
                selected_img = ImageTk.PhotoImage(img_pil)

        # الآن نحسب الجرعة حسب نوع الفحص
        if modality == "CT" and dlp > 0:
            study_desc = getattr(ds, "StudyDescription", "").lower()
            if "head" in study_desc or "brain" in study_desc:
                k = 0.0021
            elif "neck" in study_desc:
                k = 0.0059
            elif "chest" in study_desc:
                k = 0.014
            elif "abdomen" in study_desc or "pelvis" in study_desc:
                k = 0.015
            msv = dlp * k

        elif modality in ["CR", "DR", "DX", "X-RAY"] and total_dap > 0:
            k = 0.12  # Chest default; يمكن تعديله حسب المنطقة إن وُجد
            msv = total_dap * k

        key = (name, date, pid, getattr(ds, "AccessionNumber", ""))

        if key not in temp_cases:
            data_dict = {
                "Name": name,
                "Date": date,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "kFactor": k,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "PatientID": pid,
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Images": [],
                "Path": path,
                "Modality": modality,
                "Dataset": ds
            }
            temp_cases[key] = data_dict

        if selected_img:
            temp_cases[key]["Images"].append(selected_img)

        hl7_msg = convert_to_hl7_from_table(temp_cases[key])
        hl7_filename = f"{HL7_DIR}/{name}_{date.strftime('%Y%m%d')}_{pid}.hl7"
        with open(hl7_filename, "w") as f:
            f.write(hl7_msg)

    all_data.extend(temp_cases.values())

    # حساب AccumulatedDose و DosePerYear
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()


In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    grouped_files = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path, stop_before_pixels=True)
            pid = getattr(ds, "PatientID", "")
            study_date = getattr(ds, "StudyDate", "00000000")
            date_obj = datetime.strptime(study_date, "%Y%m%d") if study_date else datetime.now()
            key = (pid, date_obj)
            if key not in grouped_files:
                grouped_files[key] = []
            grouped_files[key].append(path)
        except:
            continue

    for (pid, date_obj), paths in grouped_files.items():
        name = "Unknown"
        dlp = 0
        ctdi = 0
        dap_total = 0
        modality = "Unknown"
        k = 0.015
        msv = 0
        study_desc = ""

        for path in paths:
            try:
                ds = pydicom.dcmread(path)
                name = str(getattr(ds, "PatientName", "Unknown"))
                modality = getattr(ds, "Modality", modality)
                study_desc = getattr(ds, "StudyDescription", study_desc).lower()

                if modality == "CT":
                    # لو DLP موجود مباشر
                    dlp = max(dlp, float(getattr(ds, "DLP", 0)))
                    
                    # لو في صورة Protocol تحتوي على Total DLP نستخدم OCR
                    series_desc = getattr(ds, "SeriesDescription", "").lower()
                    if "protocol" in series_desc and 'PixelData' in ds:
                        arr = ds.pixel_array
                        if arr.dtype != 'uint8':
                            arr = (arr / arr.max() * 255).astype('uint8')
                        img_pil = Image.fromarray(arr).convert("L")
                        try:
                            text = pytesseract.image_to_string(img_pil)
                            match = re.search(r"Total\s*DLP\s*[:=]?\s*(\d+\.?\d*)", text, re.IGNORECASE)
                            if match:
                                dlp = float(match.group(1))
                        except Exception as e:
                            print("OCR Error:", e)

                elif modality in ["CR", "DX", "DR", "XA", "RF", "X-Ray"]:
                    # جمع كل قيم DAP
                    dap = getattr(ds, (0x0018, 0x115E), 0)  # Image And Fluoroscopy Area Dose Product
                    try:
                        dap_total += float(dap)
                    except:
                        pass

            except Exception as e:
                print("Error reading file:", e)

        # تحديد k بناءً على وصف الدراسة
        if "head" in study_desc or "brain" in study_desc:
            k = 0.0021
        elif "neck" in study_desc:
            k = 0.0059
        elif "chest" in study_desc:
            k = 0.014
        elif "abdomen" in study_desc or "pelvis" in study_desc:
            k = 0.015
        elif "extremity" in study_desc or "limb" in study_desc:
            k = 0.0004
        else:
            k = 0.01  # fallback for general

        if modality == "CT" and dlp > 0:
            msv = dlp * k
        elif modality in ["CR", "DX", "DR", "XA", "RF", "X-Ray"] and dap_total > 0:
            msv = dap_total * k

        key = (name, date_obj.date(), pid, "")
        img = None
        if 'PixelData' in ds:
            try:
                arr = ds.pixel_array
                if arr.dtype != 'uint8':
                    arr = (arr / arr.max() * 255).astype('uint8')
                img_pil = Image.fromarray(arr).convert("L")
                img_pil.thumbnail((400, 400))
                img = ImageTk.PhotoImage(img_pil)
            except:
                pass

        if key not in temp_cases:
            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "DAP": dap_total,
                "mSv": msv,
                "kFactor": k,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "PatientID": pid,
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Images": [],
                "Path": paths[0],
                "Modality": modality,
                "Dataset": ds
            }
            temp_cases[key] = data_dict

        if img is not None:
            temp_cases[key]["Images"].append(img)

        hl7_msg = convert_to_hl7_from_table(temp_cases[key])
        hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{pid}.hl7"
        with open(hl7_filename, "w") as f:
            f.write(hl7_msg)

    all_data.extend(temp_cases.values())

    # حساب AccumulatedDose و DosePerYear
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()


In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    case_dap_totals = {}  # لتجميع DAP لحالات X-RAY

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            modality = getattr(ds, "Modality", "").upper()
            series_desc = getattr(ds, "SeriesDescription", "").lower()
            study_desc = getattr(ds, "StudyDescription", "").lower()
            patient_id = getattr(ds, "PatientID", "")
            accession = getattr(ds, "AccessionNumber", "")

            key = (name, date_obj.date(), patient_id, accession)
            if key not in temp_cases:
                temp_cases[key] = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": 0,
                    "DLP": 0,
                    "mSv": 0,
                    "kFactor": 0,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "PatientID": patient_id,
                    "Accession": accession,
                    "Images": [],
                    "Path": path,
                    "Modality": modality,
                    "Dataset": ds
                }

            # حساب DLP من CT
            if modality == "CT":
                ctdi = float(getattr(ds, "CTDIvol", 0))
                dlp = float(getattr(ds, "DLP", 0))
                k = 0.015
                
                # لو في صورة بروتوكول
                if "protocol" in series_desc and 'PixelData' in ds:
                    arr = ds.pixel_array
                    if arr.max() > 0:
                        arr = (arr / arr.max() * 255).astype('uint8')
                        img_pil = Image.fromarray(arr).convert("L")
                        try:
                            text = pytesseract.image_to_string(img_pil)
                            match = re.search(r"Total\\s*DLP\\s*[:=]?\\s*(\\d+\\.?\\d*)", text, re.IGNORECASE)
                            if match:
                                dlp = float(match.group(1))
                        except Exception as e:
                            print("OCR Error:", e)
                
                if "head" in study_desc:
                    k = 0.0021
                elif "neck" in study_desc:
                    k = 0.0059
                elif "chest" in study_desc:
                    k = 0.014
                elif "abdomen" in study_desc or "pelvis" in study_desc:
                    k = 0.015

                msv = dlp * k
                temp_cases[key]["CTDIvol"] = ctdi
                temp_cases[key]["DLP"] = dlp
                temp_cases[key]["kFactor"] = k
                temp_cases[key]["mSv"] = msv

            # حساب DAP من X-RAY/CR/DR
            elif modality in ["CR", "DR", "DX", "XRAY"]:
                dap = ds.get((0x0018, 0x115E), None)
                if dap:
                    if key not in case_dap_totals:
                        case_dap_totals[key] = 0.0
                    case_dap_totals[key] += float(dap.value)

                    # عامل التحويل حسب المنطقة (كمثال)
                    if "chest" in study_desc:
                        k = 0.17
                    elif "abdomen" in study_desc:
                        k = 0.2
                    elif "pelvis" in study_desc:
                        k = 0.25
                    else:
                        k = 0.2  # default

                    msv = case_dap_totals[key] * k
                    temp_cases[key]["DLP"] = case_dap_totals[key]  # هنا DLP = DAP Total
                    temp_cases[key]["kFactor"] = k
                    temp_cases[key]["mSv"] = msv

            # الصورة إن وجدت
            img = None
            if 'PixelData' in ds:
                arr = ds.pixel_array
                if arr.max() > 0:
                    arr = (arr / arr.max() * 255).astype('uint8')
                    img_pil = Image.fromarray(arr).convert("L")
                    img_pil.thumbnail((400, 400))
                    img = ImageTk.PhotoImage(img_pil)

            if img is not None:
                temp_cases[key]["Images"].append(img)

            hl7_msg = convert_to_hl7_from_table(temp_cases[key])
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{patient_id}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())

    # الحسابات التراكمية
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()
